In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


In [21]:
ratings = pd.read_csv(r"ml-100k\u.data", sep="\t",
                      names=["userId", "movieId", "rating", "timestamp"])

ratings = ratings.drop("timestamp", axis=1)

In [22]:
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

In [23]:
user_item_matrix = train.pivot_table(index="userId", columns="movieId", values="rating").fillna(0)

In [24]:
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, 
                                  index=user_item_matrix.columns, 
                                  columns=user_item_matrix.columns)

def recommend_item_based(user_id, k=10):
    user_ratings = user_item_matrix.loc[user_id]
    watched_movies = user_ratings[user_ratings > 0].index
    
    scores = item_similarity_df[watched_movies].sum(axis=1)
    scores = scores.drop(watched_movies, errors="ignore")
    return scores.sort_values(ascending=False).head(k)

In [25]:
svd = TruncatedSVD(n_components=50, random_state=42)   
matrix_svd = svd.fit_transform(user_item_matrix)
pred_ratings = np.dot(matrix_svd, svd.components_)
pred_df = pd.DataFrame(pred_ratings, 
                       index=user_item_matrix.index, 
                       columns=user_item_matrix.columns)

def recommend_svd(user_id, k=10):
    user_pred = pred_df.loc[user_id]
    watched = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    unseen = [m for m in user_item_matrix.columns if m not in watched]
    return user_pred.loc[unseen].sort_values(ascending=False).head(k)

In [26]:
def precision_at_k(user_id, recs, k=5, threshold=4):
    actual_high = test[(test.userId == user_id) & (test.rating >= threshold)].movieId
    top_k = recs.head(k).index
    hits = len(set(top_k) & set(actual_high))
    return hits / k if k > 0 else 0

In [27]:
user_id = 10

item_recs = recommend_item_based(user_id, k=10)
svd_recs = recommend_svd(user_id, k=10)

print("Item-based Recommendations:", item_recs.index.tolist())
print("SVD Recommendations:", svd_recs.index.tolist())

print("Precision@5 (Item-based):", precision_at_k(user_id, item_recs, k=5))
print("Precision@5 (SVD):", precision_at_k(user_id, svd_recs, k=5))

Item-based Recommendations: [174, 204, 191, 423, 172, 173, 210, 182, 211, 79]
SVD Recommendations: [483, 182, 514, 523, 12, 143, 582, 508, 432, 204]
Precision@5 (Item-based): 0.4
Precision@5 (SVD): 0.6
